# DreamBooth

In [ ]:
#@title One-Click Setup

# remove sample data provided by Google Colab
!rm -r /content/sample_data

# download necessary files
!git clone https://github.com/rmyj/dreambooth.git

!pip install -r /content/dreambooth/requirements.txt

# download validation set - designed for Google Colab
%cd /content/
for i in range(173):
  i = i + 1
  url = f"https://github.com/rmyj/txt2img/raw/main/assets/sd-v1-4.zip.{i:03}"
  !wget $url
# concatenate all part files to one zip
!cat /content/sd-v1-4.zip* > /content/sd-v1-4.zip
!rm /content/sd-v1-4.zip.*
# repair combination
!zip -FF /content/sd-v1-4.zip --out /content/sd-v1-4-full.zip
!rm /content/sd-v1-4.zip
# unzip combination
!unzip /content/sd-v1-4-full.zip
!rm /content/sd-v1-4-full.zip

In [2]:
#@title Upload photos of your subject
from google.colab import files
files.upload()

!mv /content/*.PNG /content/dreambooth/data/subject/
!mv /content/*.png /content/dreambooth/data/subject/
!mv /content/*.jpg /content/dreambooth/data/subject/

Saving image (1).png to image (1).png
Saving image (2).png to image (2).png
Saving image (3).png to image (3).png
Saving image (4).png to image (4).png
Saving image (5).png to image (5).png
Saving image (6).png to image (6).png
Saving image (7).png to image (7).png
Saving image (8).png to image (8).png
Saving image (9).png to image (9).png
Saving image (10).png to image (10).png
Saving image (11).png to image (11).png
Saving image (12).png to image (12).png
Saving image (13).png to image (13).png
Saving image (14).png to image (14).png
Saving image (15).png to image (15).png
Saving image (16).png to image (16).png
Saving image (17).png to image (17).png
Saving image (18).png to image (18).png
Saving image (19).png to image (19).png
Saving image (20).png to image (20).png
Saving image (21).png to image (21).png
Saving image (22).png to image (22).png
Saving image (23).png to image (23).png
Saving image (24).png to image (24).png
Saving image (25).png to image (25).png
Saving image (26).

In [7]:
#@title Train
!accelerate launch /content/dreambooth/scripts/train_dreambooth.py \
  --pretrained_model_name_or_path="/content/sd-v1-4/"  \
  --instance_data_dir="/content/dreambooth/data/subject/" \
  --output_dir="/content/dreambooth/models/output" \
  --instance_prompt="a photo of fefefefefefefe person" \
  --resolution=512 \
  --train_batch_size=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=2000

	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
Traceback (most recent call last):
  File "/content/dreambooth/scripts/train_dreambooth.py", line 795, in <module>
    main(args)
  File "/content/dreambooth/scripts/train_dreambooth.py", line 609, in main
    lr_scheduler = get_scheduler(
TypeError: get_scheduler() got an unexpected keyword argument 'num_cycles'
Traceback (most recent call last):
  File "/usr/local/bin/accelerate", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.8/dist-packages/accelerate/commands/accelerate_cli.py", line 45, in main
    args.func(args)
 

## convert to .ckpt

In [ ]:
#@markdown Download script
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py

In [ ]:
#@markdown Run conversion.
input_path = "/home/jovyan/work/models/sd-v1-5-inpainting"
output_path = "/home/jovyan/work/models/sd-v1-5-inpainting.ckpt"

!python3 scripts/convert_diffusers_to_original_stable_diffusion.py --model_path $input_path --checkpoint_path $output_path
print(f"[*] Converted ckpt saved at {output_path}")

***
# Inference

## Setup

In [ ]:
from PIL import Image
from diffusers import StableDiffusionPipeline
import random
import uuid
import torch

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

# set variables for inference
model_id = "dreambooth"
model_dir = "/content/models/" + model_id + "/"
pipe = StableDiffusionPipeline.from_pretrained(model_dir, torch_dtype=torch.float16).to("cuda")

# to disable the safety checker
pipe.safety_checker = lambda images, clip_input: (images, False)

## Generate

## 512x512 single image synthesis

In [ ]:
prompt = "oil painting of a landscape, midjourneyart style"
num_inference_steps=68
guidance_scale=7.5
width=512
height=512

seed = uuid.uuid4().int & (1<<64)-1
seed = 420

g_cuda = torch.Generator("cuda").manual_seed(seed)

print(f"{prompt}-{seed}")
image = pipe(prompt, height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]
image.save(f"/home/jovyan/work/output/{prompt}-{seed}-md.png")
image

## 768x768 single image synthesis

note 768 is halfway between 512 and 1024

In [ ]:
prompt = "digital painting of (sks jeremy), nebula, rainbow, psychedelic, tiki, concept art, by Greg Rutkowski and Wlop"
height = 768
width = 768

image = pipe(prompt, height=height, width=width, num_inference_steps=50, guidance_scale=7.5).images[0]

image.save(f"/home/jovyan/work/output/{prompt}-{str(uuid.uuid4())}.png")
image

## facebook cover photo image synthesis

In [ ]:
prompt = "digital painting of five headed (sks jeremy), medusa, neon magma AI, colourful rainbow supernova, in the style of Wlop and Greg Rutkowski"
height = 312
width = 824 # needs to be cropped down to 820

for _ in range(100):

    # can set seed here
    seed = uuid.uuid4().int & (1<<64)-1
    g_cuda = torch.Generator("cuda").manual_seed(seed)

    image = pipe(prompt, height=height, width=width, num_inference_steps=68, guidance_scale=7.5, generator=g_cuda).images[0]

    image.save(f"/home/jovyan/work/output/{prompt}-{seed}.png")
image

## brute force prompt design

In [ ]:
import random

# even more inspiration https://www.betchashesews.com/midjourney-portraits/
# get ideas from https://lexica.art/?
# check idea on https://rom1504.github.io/clip-retrieval/?back=https%3A%2F%2Fknn5.laion.ai&index=laion5B&useMclip=false&query=james+gurney

mediums = open('/home/jovyan/work/prompt/mediums.txt', 'r').read().splitlines()
mediums = mediums + ["very very old portrait", "Venetian portrait", "Utrecht Caravaggism portrait", "Caravaggisti oil portrait", "rococo oil portrait", "renaissance oil portrait", "classicism oil portrait", "Contemporary oil portrait", "abstract expressionism portrait", "high resolution portrait", "17th century portrait", "18th century portrait", "watercolour portrait", "baroque oil portrait", "full face epic portrait", "digital portrait", "oil portrait", "portrait", "realism portrait", "photorealism portrait", "Surrealism portrait", "Impressionism portrait", "Expressionism portrait"]
# of 
adjectives = ["good", "fat", "angelic", "great", "powerful", "noble", "regal", "godlike", "three headed", "handsome", "graceful", "lovely", "smart", "intelligent", "majestic", "suave", "good-looking", "elegant", "attractive", "beautiful", "happy", "smiling", "colourful", "laughing"]

subject = ["sks jeremy", "sks emma"]

# as a...
roles = ["an emo", "a grand wizard", "a wizard", "a goth", "a robot", "a child", "an old man", "a grandfather", "a dictator", "james bond", "robin hood", "a skeleton", "he-man", "barney the dinosaur", "ronald mcdonald", "colonel sanders", "kfc colonel", "an alien", "the hulk", "a cricket player", "a rugby player", "an mma fighter", "a boxer", "the president", "a writer", "a god", "a diety", "the messiah", "a tibetian buddhist monk", "a korean dictator", "a philanthropist", "a banker", "a japanese samurai", "a baby", "zeus", "thor", "Dionysus", "a sumo", "a prince", "a monk", "an arab", "a bodybuilder", "a barbarian", "a king", "a lord", "a pharaoh", "a billionaire", "a monkey", "a pimp", "a nigerian prince"]

# with...
characters = ["john cena", "joe rogan"]

# with middle words
middle = ["this is fine meme dog", "snow", "nuclear mushroom", "a million colourful balloons", "evga FTW3 RTX 3090", "rapunzel hair", "flowers in hair", "macaw", "knightly armour", "shoe on head", "book on head", "high-key", "longest moustache in the world", "casting a spell", "in varrock", "in lumbridge", "with a fire staff", "shoe on head", "colourful bowtie", "colourful scarf", "colourful party hat", "osrs party hat", "runescape party hat", "rune scimitar", "osrs graphics", "2007scape", "old master", "old masters", "louvre collection", "museum collection", "traditional chinese wedding dress", "Todos Santos Cuchumatán", "wearing Sari", "wearing gho", "Conical hat", "Sardinian traditional dress", "keffiyeh", "shemagh" ,"ghutrah", "colourful traditional clothing", "wearing long royal robe", "sequins", "academic", "jeweled", "detailed face", "fashion photography", "male model", "shallow depth of field", "morrowind", "beautiful composition", "chin-strap facial hair", "colourful facial hair", "colourful hair", "painting in museum", "sharp focus", "realistic", "intricate", "octane render", "trending artstation", "monochrome", "extreme detail", "Utrecht Caravaggism", "on a magic carpet", "on horseback", "glitter", "neon lights", "long hair", "inferno", "vivid colors", "mycelium", "colourful smoke", "colourful clouds", "quantum physics", "rembrandt lighting", "tesla coil", "plasma", "happy accidents", "tekken character", "street fighter character", "atari graphics", "tempera", "Ukiyo-e", "cumulus", "portrait", "being fed grapes", "looking at camera", "isometric view", "full body", "full body portrait", "full-body portrait", "freeze frame shot", "from behind", "aerial shot", "bird's eye shot", "😂", "❤️", "🤣", "🔥", "😍", "🥰", "🙏", "✨", "🔞💦🍆😳👄🍑🚿😈👅", "World Heritage Site", "sunbeam", "crepuscular rays", "god rays", "asperitas clouds", "かわいい", "oneshota", "Dungeons & Dragons", "patronus", "Lovecraftian", "daguerreotype", "Gorillaz", "prismatic", "bokeh", "raining", "far from camera", "vibrant", "epic", "volumetric lighting", "jellyfish from finding nemo", "disney", "pixar", "3D render", "boutonnière", "balloons from UP", "HDR", "holi", "turban", "with a thousand arms", "mushroom helmet", "reflection", "transparent", "british explorers", "smoking a cigar", "drinking a cocktail", "summoning a taxi", "german traditional dress", "santa hat", "noita", "Cel Shaded", "Margit, the Fell Omen", "Morgott, the Omen King", "bandholz beard", "Rennala, Queen of the Full Moon", "elden beast", "elden ring", "fel fire", "blue fire", "smooth", "taniwha", "surrounded by sheep", "Silver fern", "tui", "wood pigeon", "kereru", "pounamu", "kiwiana", "pohutawaka", "kowhai", "surrounded by new zealand birds", "surrounded by new zealand bush", "surrounded by clouds", "sharp focus", "Picasso's African Period", "Picasso's Blue Period, fine art", "bladerunner", "baroque element", "baroque elements", "complimentry colours", "river", "surrounded by butterflies", "surrounded by flowers", "unreal engine 5", "lightsaber", "hacker", "bowtie", "Barong", "balinese mask", "tribal mask", "african mask", "award winning photography", "award winning", "deviantart", "surrounded by ice", "frozen", "divine", "surrounded by clifford the big red dog", "monocle", "top hat", "meme", "kimono", "Windsor glasses", "surrounded by bunnies", "surrounded by piglets", "surrounded by pandas", "surrounded by frogs", "blindfolded", "gold chain", "bling", "surrounded by puppies", "surrounded by dogs", "surrounded by kittens", "surrounded by cats", "LSD visuals", "DMT fractal patterns", "ornate", "hallucinogen", "mutagen", "mutated", "coronation robes 1701", "masterpiece", "fu manchu moustache", "star-burns", "muttonchops", "captain hook moustache", "salvador dali moustache", "animorphs", "on fire", "barbaric", "hyper detailed", "ultra sharp", "sunny", "overwatch character", "highly detailed", "detailed face", "papakha hat", "papakha woolen hat", "tamoko", "cinematic lighting", "golden", "rich", "exotic", "spicy", "fruity", "yoked", "jacked", "holy", "royal", "bearded", "neon", "waterfall", "whirlpool", "electric", "cyberpunk", "futuristic", "pink", "opal", "iridescent", "vibrant", "high society", "artistic", "artstation", "rainbow supernova", "masculine", "rule of thirds", "beautiful", "8k", "4k", "hooded cloak", "glowing", "french moustache", "viking beard", "handlebar moustache", "imperial beard", "Ultra realistic", "heavy brushstrokes", "crisp", "sharp", "ultra sharp", "high resolution", "breathtaking", "trending on artstation", "bioluminous", "funky", "medusa", "supernova", "nebula", "AI", "vibrant", "white dreadlocks", "rainbow"]

# middle should be split into subjects and properties
# add one or more properties but only ever add one subject

portrait_artists = []
landscape_artists = []
artists = portrait_artists

# from artist(s)
artists = ["Jean Auguste Dominique Ingres", "Georges Seurat", "Sandro Botticelli", "bob ross", "makoto shinkai", "Adam Rex", "Genzoman", "Rebecca Guay", "Igor Kieryluk", "Richard Wright", "Veronique Meignaud", "Loish", "Jan Vermeer", "Akira Toriyama", "Hayao Miyazaki", "tuomas korpi", "Jean Giraud", "carl spitzweg", "Konstantin Turovec", "Botos Vlad", "Alex Horley", "Anato Finnstark", "Josh Dykgraaf", "Katsushika Hokusai", "hokusai", "jan van eyck", "max ernst", "gaston bussiere", "dang my linh", "Alex Alexandrov", "Ralph Horsley", "Tian Zi", "Peter Xiao", "Leonardo da Vinci", "Zdzisław Beksiński", "Alex Grey", "Dan Mumford", "Raymond Swanland", "Rembrandt", "Victo Ngai", "Artem Chebokha", "Aleksander Rostov", "Jen Zee", "Stephen Bliss", "Craig Mullins", "Jakub Rozalski", "James Jean", "James Gurney", "Gregory Manchess", "Studio Ghibli", "Sachin Teng", "Huang Guangjian", "Takato Yomamoto", "Conrad Roset", "Claude Monet", "Vincent Van Gogh", "Drew Struzan", "Konstantin Korovin", "Daniel F. Gerhartz", "John Howe", "Hsiao Ron Cheng", "John James Audubon", "Wlop", "Greg Rutkowski", "Rutkowski", "grzegorz Rutkowski", "Giuseppe Arcimboldo", "Yusuke Murata", "Beeple", "Mike Winkelmann", "Simon Stalenhag", "Casey Weldon", "Artgerm", "Alphonse Mucha"]
portrait_artists = open('/home/jovyan/work/prompt/portrait_artists.txt', 'r').read().splitlines()
artists = artists + portrait_artists

def generate_prompt():
    
    prompt = ""
    
    # add one medium from mediums
    prompt = prompt + mediums[random.randint(0, len(mediums) - 1)]
    
    # add "of"
    prompt = prompt + " of "
    
    # add one or more from adjectives
    temp = random.randint(0, 0)
    for _ in range(temp): # inclusive
        prompt = prompt + adjectives[random.randint(0, len(adjectives) - 1)] + ", "
        
    if temp != 0:
        prompt = prompt[:-2]
        prompt = prompt + " "
    
    # add subject with random number of surrounding "(" and ")"
    num_brackets = random.randint(0, 10) # inclusive
    num_brackets = 0
    for _ in range(num_brackets):
        prompt = prompt + "("
    prompt = prompt + subject[0] # 0 for sks jeremy
    for _ in range(num_brackets):
        prompt = prompt + ")"
        
    # one in three chance to add "as a ___"
    rand_roles = random.randint(1, 3)
    if rand_roles == 0: # inclusive
        prompt = prompt + " as "
        prompt = prompt + roles[random.randint(0, len(roles) - 1)] + ", "
    else:
        prompt = prompt + ", "
    
    if rand_roles != 1:
        # one in three chance to add " beside " to add a second character to the image
        if random.randint(0, 0) == 1:
            prompt = prompt[:-2]
            prompt = prompt + " beside "
            for _ in range(num_brackets):
                prompt = prompt + "("
            prompt = prompt + characters[random.randint(0, len(characters) - 1)]
            for _ in range(num_brackets):
                prompt = prompt + ")"
            prompt = prompt + ", "
        
    prompt = prompt[:-2]
    prompt = prompt + ", "
            
    # add one or more from middle
    for _ in range(random.randint(0, 2)): # inclusive
        prompt = prompt + middle[random.randint(0, len(middle) - 1)]
        prompt = prompt + ", "
    
    # add "in the style of"
    prompt = prompt + "in the style of "
    
    # add one or more artists to style from artists
    for _ in range(random.randint(1, 3)): # inclusive
        prompt = prompt + artists[random.randint(0, len(artists) - 1)] + ", "
        
    prompt = prompt[:-2]
    
    return prompt

height = 720
width = 1280

width = 1320

while True:
    # can set seed here
    seed = uuid.uuid4().int & (1<<64)-1
    g_cuda = torch.Generator("cuda").manual_seed(seed)
    
    prompt = generate_prompt()
    
    num_inference_steps=199
    
    # just in case the prompt is longer than what can be saved
    while len(f"{prompt}-{seed}-{num_inference_steps}.png") > 248: # linux can handle 255, jupyter can handle 248
        print("\"" + prompt, "\" is too long! regenerating...")
        prompt = generate_prompt()
        
    print(f"{prompt}-{seed}")
    image = pipe(prompt, height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=7.5, generator=g_cuda).images[0]
    
    image.save(f"/home/jovyan/work/output/{prompt}-{seed}-{num_inference_steps}.png")
    print(f"saved to /home/jovyan/work/output/")
    print()
image

## brute force prompt design v2

In [ ]:
import random
import time

# even more inspiration https://www.betchashesews.com/midjourney-portraits/
# get ideas from https://lexica.art/?
# check idea on https://rom1504.github.io/clip-retrieval/?back=https%3A%2F%2Fknn5.laion.ai&index=laion5B&useMclip=false&query=james+gurney

# read prompt ingredients from .txt file
mediums = open('/home/jovyan/work/prompt/mediums.txt', 'r').read().splitlines()
subject = ["sks jeremy"]
portrait_artists = open('/home/jovyan/work/prompt/portrait_artists.txt', 'r').read().splitlines()

middle = ["mandelbulb fractal", "mandelbrot set", "antimatter", "gravity", "futuristic", "robotic", "photoshopped", "bright neon colors", "glacier", "glaciation", "aesthetically pleasing", "fireworks", "visually stunning", "heavenly glow", "heavenly aura", "concept art portrait", "4k resolution", "8k resolution", "4k res", "8k res", "assassin's creed costume", "Thermoluminescence", "aztec god", "psychedelic rainbow magic", "sunlight streaming through clouds", "wispy clouds", "long hair", "long hairstyle", "bald", "wavy hair", "this is fine meme dog", "icy", "snow", "nuclear mushroom", "a million colourful balloons", "evga FTW3 RTX 3090", "rapunzel hair", "flowers in hair", "macaw", "knightly armour", "shoe on head", "book on head", "high-key", "longest moustache in the world", "casting a spell", "in varrock", "in lumbridge", "with a fire staff", "shoe on head", "colourful bowtie", "colourful scarf", "colourful party hat", "osrs party hat", "runescape party hat", "rune scimitar", "osrs graphics", "2007scape", "old master", "old masters", "louvre collection", "museum collection", "traditional chinese wedding dress", "Todos Santos Cuchumatán", "wearing Sari", "wearing gho", "Conical hat", "Sardinian traditional dress", "keffiyeh", "shemagh" ,"ghutrah", "colourful traditional clothing", "wearing long royal robe", "sequins", "academic", "jeweled", "detailed face", "fashion photography", "male model", "shallow depth of field", "morrowind", "beautiful composition", "chin-strap facial hair", "colourful facial hair", "colourful hair", "painting in museum", "sharp focus", "realistic", "intricate", "octane render", "trending artstation", "monochrome", "extreme detail", "Utrecht Caravaggism", "on a magic carpet", "on horseback", "glitter", "neon lights", "long hair", "inferno", "vivid colors", "mycelium", "colourful smoke", "colourful clouds", "quantum physics", "rembrandt lighting", "tesla coil", "plasma", "happy accidents", "tekken character", "street fighter character", "atari graphics", "tempera", "Ukiyo-e", "cumulus", "portrait", "being fed grapes", "looking at camera", "isometric view", "full body", "full body portrait", "full-body portrait", "freeze frame shot", "from behind", "aerial shot", "bird's eye shot", "😂", "❤️", "🤣", "🔥", "😍", "🥰", "🙏", "✨", "🔞💦🍆😳👄🍑🚿😈👅", "World Heritage Site", "sunbeam", "crepuscular rays", "god rays", "asperitas clouds", "かわいい", "oneshota", "Dungeons & Dragons", "patronus", "Lovecraftian", "daguerreotype", "Gorillaz", "prismatic", "bokeh", "raining", "far from camera", "vibrant", "epic", "volumetric lighting", "jellyfish from finding nemo", "disney", "pixar", "3D render", "boutonnière", "balloons from UP", "HDR", "holi", "turban", "with a thousand arms", "mushroom helmet", "reflection", "transparent", "british explorers", "smoking a cigar", "drinking a cocktail", "summoning a taxi", "german traditional dress", "santa hat", "noita", "Cel Shaded", "Margit, the Fell Omen", "Morgott, the Omen King", "bandholz beard", "Rennala, Queen of the Full Moon", "elden beast", "elden ring", "fel fire", "blue fire", "smooth", "taniwha", "surrounded by sheep", "Silver fern", "tui", "wood pigeon", "kereru", "pounamu", "kiwiana", "pohutawaka", "kowhai", "surrounded by new zealand birds", "surrounded by new zealand bush", "surrounded by clouds", "sharp focus", "Picasso's African Period", "Picasso's Blue Period", "fine art", "bladerunner", "baroque element", "baroque elements", "complimentry colours", "river", "surrounded by butterflies", "surrounded by flowers", "unreal engine 5", "lightsaber", "hacker", "bowtie", "Barong", "balinese mask", "tribal mask", "african mask", "award winning photography", "award winning", "deviantart", "surrounded by ice", "frozen", "divine", "surrounded by clifford the big red dog", "monocle", "top hat", "meme", "kimono", "Windsor glasses", "surrounded by bunnies", "surrounded by piglets", "surrounded by pandas", "surrounded by frogs", "blindfolded", "gold chain", "bling", "surrounded by puppies", "surrounded by dogs", "surrounded by kittens", "surrounded by cats", "LSD visuals", "DMT fractal patterns", "ornate", "hallucinogen", "mutagen", "mutated", "coronation robes 1701", "masterpiece", "fu manchu moustache", "star-burns", "muttonchops", "captain hook moustache", "salvador dali moustache", "animorphs", "on fire", "barbaric", "hyper detailed", "ultra sharp", "sunny", "overwatch character", "highly detailed", "detailed face", "papakha hat", "papakha woolen hat", "tamoko", "cinematic lighting", "golden", "rich", "exotic", "spicy", "fruity", "yoked", "jacked", "holy", "royal", "bearded", "neon", "waterfall", "whirlpool", "electric", "cyberpunk", "futuristic", "pink", "opal", "iridescent", "vibrant", "high society", "artistic", "artstation", "rainbow supernova", "masculine", "rule of thirds", "beautiful", "8k", "4k", "hooded cloak", "glowing", "french moustache", "viking beard", "handlebar moustache", "imperial beard", "Ultra realistic", "heavy brushstrokes", "crisp", "sharp", "ultra sharp", "high resolution", "breathtaking", "trending on artstation", "bioluminous", "funky", "medusa", "supernova", "nebula", "AI", "vibrant", "white dreadlocks", "rainbow"]

def generate_prompt():
    
    prompt = ""
    
    # add one medium from mediums
    prompt = prompt + mediums[random.randint(0, len(mediums) - 1)]
    prompt = "old master"
    
    prompt = prompt + " portrait of sks jeremy"
    
    # adds " portrait of sks jeremy" to the prompt
    #prompt = prompt + " portrait of " + subject[0]
    
    # additives for memes
    prompt = prompt + " wearing a hat, perfect smile, french moustache, "
    
    # add some stuff in the middle
    for _ in range(random.randint(0, 2)):
        prompt = prompt + middle[random.randint(0, len(middle) - 1)] + ", "
        
    prompt = prompt[:-2]
    
    # additives for quality
    #prompt = prompt + "photorealistic, very detailed, trending on artstation, "
    prompt = prompt + ", over saturated, ultra realistic, very detailed, trending on artstation, 4k, "
    
    # add "in the style of"
    prompt = prompt + ""
    
    # add one or more artists to style from artists
    for _ in range(random.randint(1, 1)): # inclusive
        #prompt = prompt + portrait_artists[random.randint(0, len(portrait_artists) - 1)] + ", "
        prompt = prompt + "greg rutkowski, "
        
        # try fixing it to Craig Mullins
        
    prompt = prompt[:-2]
    
    return prompt

height = 720
width = 1320

while True:
    # can set seed here
    seed = uuid.uuid4().int & (1<<64)-1
    g_cuda = torch.Generator("cuda").manual_seed(seed)
    
    prompt = generate_prompt()
    
    num_inference_steps=68
    
    # just in case the prompt is longer than what can be saved
    save = 0
    while len(f"{prompt}-{seed}-{num_inference_steps}.png") > 234: # linux can handle 255, jupyter can handle 248 or maybe 247 or maybe 246 but cant handle 235??
        print("prompt was too long at " + str(len(f"{prompt}-{seed}-{num_inference_steps}.png")) + " characters long! regenerating...")
        prompt = generate_prompt()
        time.sleep(0.01667) # will keep trying at the rate of 60 per second
        
    print(f"{prompt}-{seed}-{num_inference_steps}")
    image = pipe(prompt, height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=7.5, generator=g_cuda).images[0]
    
    image.save(f"/home/jovyan/work/output/{prompt}-{seed}-{num_inference_steps}.png")
    print(f"saved to /home/jovyan/work/output/")
    print()
image

## brute force v3
mainly for working with the new midjourney-jeremy models

In [ ]:
height=512
width=512

#width=1280
#height=720

from PIL.PngImagePlugin import PngImageFile, PngInfo
import random
middle = ["royal gardens", "garden of eden", "eden", "mandelbulb fractal", "mandelbrot set", "antimatter", "gravity", "futuristic", "robotic", "photoshopped", "bright neon colors", "glacier", "glaciation", "aesthetically pleasing", "fireworks", "visually stunning", "heavenly glow", "heavenly aura", "concept art portrait", "4k resolution", "8k resolution", "4k res", "8k res", "assassin's creed costume", "Thermoluminescence", "aztec god", "psychedelic rainbow magic", "sunlight streaming through clouds", "wispy clouds", "long hair", "long hairstyle", "bald", "wavy hair", "this is fine meme dog", "icy", "snow", "nuclear mushroom", "a million colourful balloons", "evga FTW3 RTX 3090", "rapunzel hair", "flowers in hair", "macaw", "knightly armour", "shoe on head", "book on head", "high-key", "longest moustache in the world", "casting a spell", "in varrock", "in lumbridge", "with a fire staff", "shoe on head", "colourful bowtie", "colourful scarf", "colourful party hat", "osrs party hat", "runescape party hat", "rune scimitar", "osrs graphics", "2007scape", "old master", "old masters", "louvre collection", "museum collection", "traditional chinese wedding dress", "Todos Santos Cuchumatán", "wearing Sari", "wearing gho", "Conical hat", "Sardinian traditional dress", "keffiyeh", "shemagh" ,"ghutrah", "colourful traditional clothing", "wearing long royal robe", "sequins", "academic", "jeweled", "detailed face", "fashion photography", "male model", "shallow depth of field", "morrowind", "beautiful composition", "chin-strap facial hair", "colourful facial hair", "colourful hair", "painting in museum", "sharp focus", "realistic", "intricate", "octane render", "trending artstation", "monochrome", "extreme detail", "Utrecht Caravaggism", "on a magic carpet", "on horseback", "glitter", "neon lights", "long hair", "inferno", "vivid colors", "mycelium", "colourful smoke", "colourful clouds", "quantum physics", "rembrandt lighting", "tesla coil", "plasma", "happy accidents", "tekken character", "street fighter character", "atari graphics", "tempera", "Ukiyo-e", "cumulus", "portrait", "being fed grapes", "looking at camera", "isometric view", "full body", "full body portrait", "full-body portrait", "freeze frame shot", "from behind", "aerial shot", "bird's eye shot", "😂", "❤️", "🤣", "🔥", "😍", "🥰", "🙏", "✨", "🔞💦🍆😳👄🍑🚿😈👅", "World Heritage Site", "sunbeam", "crepuscular rays", "god rays", "asperitas clouds", "かわいい", "oneshota", "Dungeons & Dragons", "patronus", "Lovecraftian", "daguerreotype", "Gorillaz", "prismatic", "bokeh", "raining", "far from camera", "vibrant", "epic", "volumetric lighting", "jellyfish from finding nemo", "disney", "pixar", "3D render", "boutonnière", "balloons from UP", "HDR", "holi", "turban", "with a thousand arms", "mushroom helmet", "reflection", "transparent", "british explorers", "smoking a cigar", "drinking a cocktail", "summoning a taxi", "german traditional dress", "santa hat", "noita", "Cel Shaded", "Margit, the Fell Omen", "Morgott, the Omen King", "bandholz beard", "Rennala, Queen of the Full Moon", "elden beast", "elden ring", "fel fire", "blue fire", "smooth", "taniwha", "surrounded by sheep", "Silver fern", "tui", "wood pigeon", "kereru", "pounamu", "kiwiana", "pohutawaka", "kowhai", "surrounded by new zealand birds", "surrounded by new zealand bush", "surrounded by clouds", "sharp focus", "Picasso's African Period", "Picasso's Blue Period", "fine art", "bladerunner", "baroque element", "baroque elements", "complimentry colours", "river", "surrounded by butterflies", "surrounded by flowers", "unreal engine 5", "lightsaber", "hacker", "bowtie", "Barong", "balinese mask", "tribal mask", "african mask", "award winning photography", "award winning", "deviantart", "surrounded by ice", "frozen", "divine", "surrounded by clifford the big red dog", "monocle", "top hat", "meme", "kimono", "Windsor glasses", "surrounded by bunnies", "surrounded by piglets", "surrounded by pandas", "surrounded by frogs", "blindfolded", "gold chain", "bling", "surrounded by puppies", "surrounded by dogs", "surrounded by kittens", "surrounded by cats", "LSD visuals", "DMT fractal patterns", "ornate", "hallucinogen", "mutagen", "mutated", "coronation robes 1701", "masterpiece", "fu manchu moustache", "star-burns", "muttonchops", "captain hook moustache", "salvador dali moustache", "animorphs", "on fire", "barbaric", "hyper detailed", "ultra sharp", "sunny", "overwatch character", "highly detailed", "detailed face", "papakha hat", "papakha woolen hat", "tamoko", "cinematic lighting", "golden", "rich", "exotic", "spicy", "fruity", "yoked", "jacked", "holy", "royal", "bearded", "neon", "waterfall", "whirlpool", "electric", "cyberpunk", "futuristic", "pink", "opal", "iridescent", "vibrant", "high society", "artistic", "artstation", "rainbow supernova", "masculine", "rule of thirds", "beautiful", "8k", "4k", "hooded cloak", "glowing", "french moustache", "viking beard", "handlebar moustache", "imperial beard", "Ultra realistic", "heavy brushstrokes", "crisp", "sharp", "ultra sharp", "high resolution", "breathtaking", "trending on artstation", "bioluminous", "funky", "medusa", "supernova", "nebula", "AI", "vibrant", "white dreadlocks", "rainbow"]

while True:
    # can set seed here
    seed = uuid.uuid4().int & (1<<64)-1
    g_cuda = torch.Generator("cuda").manual_seed(seed)

    # for later reference because you will probably forget jeremy
    #
    # but this is using the model_id = "/home/jovyan/work/models/jeremy-midjourney-style-2500-v4/"
    # and that model was trained using the images from "jeremy-v3-midjourney-style-v3"
    
    # oh my god i did it - the midjourney style is here baby
    prompt = "oil painting of (sks person), large french moustache, heavy brushstrokes, dramatic lighting, very detailed, insane detail, trending on artstation, 4k, craig mullins, greg rutkowski"
    
    # another god-tier prompt with this model
    prompt = "oil painting of sks person, french moustache, heavy brushstrokes, beautiful eyes, surrounded by exploding nebula, dramatic lighting, very detailed, insanely detailed, trending on artstation, 4k, craig mullins, greg rutkowski"
    
    # really good potential too
    prompt = "oil painting of sks person, french moustache, heavy brushstrokes, beautiful eyes, surrounded by exploding nebula, interdimensional, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski"

    # wow - creates human features in space formations - insanely cool
    prompt = "oil painting of sks person, french moustache, heavy brushstrokes, beautiful eyes, whirlpool, surrounded by exploding nebula, interdimensional, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski"

    # new one these are too good - not many new features but worthy
    prompt = "oil painting of sks person, french moustache, heavy brushstrokes, beautiful eyes, whirlpool, surrounded by exploding nebula, pink, interdimensional, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski"

    # definitely potential, seems like these work REALLY well at 512x512
    prompt =  "oil painting of sks person, french moustache, heavy brushstrokes, beautiful eyes, surrounded by exploding nebula, angelic, gates of heaven, interdimensional, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski"

    # oh my god another goodie
    prompt =  "oil painting of sks person, french moustache, heavy brushstrokes, beautiful eyes, surrounded by exploding nebula, demonic, gates of hell, interdimensional, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski"

    # great potential
    prompt =  "oil painting of sks person, french moustache, heavy brushstrokes, beautiful eyes, surrounded by heaven and hell, interdimensional portal, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski"

    # god tier prompt being surrounded by puppies
    prompt =  "oil painting of sks person, french moustache, heavy brushstrokes, beautiful eyes, surrounded by puppies, interdimensional portal, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski"
    
    # a new one
    prompt = "oil painting of sks person, french moustache, heavy brushstrokes, beautiful eyes, "
    prompt = prompt + middle[random.randint(0, len(middle) - 1)]
    prompt = prompt + ", golden nebula, interdimensional portal, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski"
    
    animals = ["pigs", "kittens", "snakes", "puppies", "parrots", "sharks", "bunnies", "tropical fish", "chickens", "cows"]
    prompt =  "oil painting of sks person, french moustache, heavy brushstrokes, beautiful eyes, surrounded by " + animals[random.randint(0, len(animals) - 1)] + ", interdimensional portal, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski"
        
    num_inference_steps=68
        
    print(f"{prompt}-{seed}-{num_inference_steps}")
    guidance_scale=7.5
    image = pipe(prompt, height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, generator=g_cuda).images[0]
    
    metadata = PngInfo()
    metadata.add_text("height", str(height))
    metadata.add_text("width", str(width))
    metadata.add_text("num_inference_steps", str(num_inference_steps))
    metadata.add_text("guidance_scale", str(guidance_scale))
    metadata.add_text("prompt", prompt)
    metadata.add_text("model_id", model_id)
    metadata.add_text("seed", str(seed))
    
    image.save(f"/home/jovyan/work/output/test-{seed}.png", pnginfo=metadata)
    targetImage = PngImageFile(f"/home/jovyan/work/output/test-{seed}.png")
    
    print(f"saved to /home/jovyan/work/output/test-{seed}.png")
    print("with metadata:", targetImage.text)
    print()
    image.show()

## brute force v4 some crazy serious potential here

In [ ]:
height=512
width=512

#height=640
#width=640

#height=720
#width=720

#height=824
#width=824

height = 720
width = 1280
#width=1888

import random
from PIL.PngImagePlugin import PngImageFile, PngInfo
middle = ["royal gardens", "garden of eden", "eden", "mandelbulb fractal", "mandelbrot set", "antimatter", "gravity", "futuristic", "robotic", "photoshopped", \
          "bright neon colors", "glacier", "glaciation", "aesthetically pleasing", "fireworks", "visually stunning", "heavenly glow", "heavenly aura", \
          "4k resolution", "8k resolution", "4k res", "8k res", "assassin's creed costume", "Thermoluminescence", "aztec god", "psychedelic rainbow magic", \
          "sunlight streaming through clouds", "wispy clouds", "long hair", "long hairstyle", "bald", "wavy hair", "this is fine meme dog", "icy", "snow", \
          "nuclear mushroom", "a million colourful balloons", "evga FTW3 RTX 3090", "rapunzel hair", "flowers in hair", "macaw", "knightly armour", "shoe on head", \
          "book on head", "high-key", "longest moustache in the world", "casting a spell", "in varrock", "in lumbridge", "with a fire staff", "shoe on head", \
          "colourful bowtie", "colourful scarf", "colourful party hat", "osrs party hat", "runescape party hat", "rune scimitar", "osrs graphics", "2007scape", \
          "old master", "old masters", "louvre collection", "museum collection", "traditional chinese wedding dress", "Todos Santos Cuchumatán", "wearing Sari", \
          "wearing gho", "Conical hat", "Sardinian traditional dress", "keffiyeh", "shemagh" ,"ghutrah", "colourful traditional clothing", \
          "wearing long royal robe", "sequins", "academic", "jeweled", "detailed face", "fashion photography", "male model", "shallow depth of field", \
          "morrowind", "beautiful composition", "chin-strap facial hair", "colourful facial hair", "colourful hair", "painting in museum", "sharp focus", \
          "realistic", "intricate", "octane render", "trending artstation", "monochrome", "extreme detail", "Utrecht Caravaggism", "on a magic carpet", \
          "on horseback", "glitter", "neon lights", "long hair", "inferno", "vivid colors", "mycelium", "colourful smoke", "colourful clouds", \
          "quantum physics", "rembrandt lighting", "tesla coil", "plasma", "happy accidents", "tekken character", "street fighter character", \
          "atari graphics", "tempera", "Ukiyo-e", "cumulus", "portrait", "being fed grapes", "looking at camera", "isometric view", "full body", \
          "from behind", "aerial shot", "bird's eye shot", "😂", "❤️", "🤣", "🔥", "😍", "🥰", "🙏", "✨", "🔞💦🍆😳👄🍑🚿😈👅", "World Heritage Site", \
          "sunbeam", "crepuscular rays", "god rays", "asperitas clouds", "かわいい", "oneshota", "Dungeons & Dragons", "patronus", "Lovecraftian", "daguerreotype", \
          "Gorillaz", "prismatic", "bokeh", "raining", "far from camera", "vibrant", "epic", "volumetric lighting", "jellyfish from finding nemo", "disney", "pixar", \
          "3D render", "boutonnière", "balloons from UP", "HDR", "holi", "turban", "with a thousand arms", "mushroom helmet", "reflection", "transparent", \
          "british explorers", "smoking a cigar", "drinking a cocktail", "summoning a taxi", "german traditional dress", "santa hat", "noita", "Cel Shaded", \
          "Margit, the Fell Omen", "Morgott, the Omen King", "bandholz beard", "Rennala, Queen of the Full Moon", "elden beast", "elden ring", "fel fire", \
          "blue fire", "smooth", "taniwha", "surrounded by sheep", "Silver fern", "tui", "wood pigeon", "kereru", "pounamu", "kiwiana", "pohutawaka", \
          "kowhai", "surrounded by new zealand birds", "surrounded by new zealand bush", "surrounded by clouds", "sharp focus", "Picasso's African Period", \
          "Picasso's Blue Period", "fine art", "bladerunner", "baroque element", "baroque elements", "complimentry colours", "river", "surrounded by butterflies", \
          "surrounded by flowers", "unreal engine 5", "lightsaber", "hacker", "bowtie", "Barong", "balinese mask", "tribal mask", "african mask", \
          "award winning photography", "award winning", "deviantart", "surrounded by ice", "frozen", "divine", "surrounded by clifford the big red dog", \
          "monocle", "top hat", "meme", "kimono", "Windsor glasses", "surrounded by bunnies", "surrounded by piglets", "surrounded by pandas", \
          "surrounded by frogs", "blindfolded", "gold chain", "bling", "surrounded by puppies", "surrounded by dogs", "surrounded by kittens", \
          "surrounded by cats", "LSD visuals", "DMT fractal patterns", "ornate", "hallucinogen", "mutagen", "mutated", "coronation robes 1701", \
          "masterpiece", "fu manchu moustache", "star-burns", "muttonchops", "captain hook moustache", "salvador dali moustache", "animorphs", \
          "on fire", "barbaric", "hyper detailed", "ultra sharp", "sunny", "overwatch character", "highly detailed", "detailed face", "papakha hat", \
          "papakha woolen hat", "tamoko", "cinematic lighting", "golden", "rich", "exotic", "spicy", "fruity", "yoked", "jacked", "holy", "royal", \
          "bearded", "neon", "waterfall", "whirlpool", "electric", "cyberpunk", "futuristic", "pink", "opal", "iridescent", "vibrant", "high society", \
          "artistic", "artstation", "rainbow supernova", "masculine", "rule of thirds", "beautiful", "8k", "4k", "hooded cloak", "glowing", "french moustache", \
          "viking beard", "handlebar moustache", "imperial beard", "Ultra realistic", "heavy brushstrokes", "crisp", "sharp", "ultra sharp", "high resolution", \
          "breathtaking", "trending on artstation", "bioluminous", "funky", "medusa", "supernova", "nebula", "AI", "vibrant", "white dreadlocks", "rainbow", \
          "cotton candy", "popcorn", "coral reef", "tropical fish", "fish and chips", "foehn wall cloud"]

prompts = []
prompts.append("oil painting of (sks person), large french moustache, heavy brushstrokes, dramatic lighting, very detailed, insane detail, trending on artstation, 4k, craig mullins, greg rutkowski")
prompts.append("oil painting of sks person, french moustache, heavy brushstrokes, beautiful eyes, surrounded by exploding nebula, dramatic lighting, very detailed, insanely detailed, trending on artstation, 4k, craig mullins, greg rutkowski")
prompts.append("oil painting of sks person, french moustache, heavy brushstrokes, beautiful eyes, surrounded by exploding nebula, interdimensional, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski")
prompts.append("oil painting of sks person, french moustache, heavy brushstrokes, beautiful eyes, whirlpool, surrounded by exploding nebula, interdimensional, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski")
prompts.append("oil painting of sks person, french moustache, heavy brushstrokes, beautiful eyes, whirlpool, surrounded by exploding nebula, pink, interdimensional, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski")
prompts.append("oil painting of sks person, french moustache, heavy brushstrokes, beautiful eyes, surrounded by exploding nebula, angelic, gates of heaven, interdimensional, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski")
prompts.append("oil painting of sks person, french moustache, heavy brushstrokes, beautiful eyes, surrounded by exploding nebula, demonic, gates of hell, interdimensional, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski")
prompts.append("oil painting of sks person, french moustache, heavy brushstrokes, beautiful eyes, surrounded by heaven and hell, interdimensional portal, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski")
prompts.append("oil painting portrait of sks person, french moustache, heavy brushstrokes, beautiful eyes, whirlpool, surrounded by exploding nebula, interdimensional, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski")
prompts.append("oil painting portrait of sks person, french moustache, surrounded by colorful puppies, colorful blazer, beautiful brown eyes, heavy brushstrokes, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski")
prompts.append("oil painting portrait of sks person, french moustache, surrounded by colorful cows, colorful blazer, beautiful brown eyes, heavy brushstrokes, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski")

# use GOATED "jeremy-midjourney-style-2500-v4" with this loop 
while True:
    # can set seed here
    seed = uuid.uuid4().int & (1<<64)-1
    g_cuda = torch.Generator("cuda").manual_seed(seed)
    
    prompt = "Hello world!"

    temp = random.randint(0, 3)
    if temp == 0:
        prompt = prompts[random.randint(0, len(prompts) - 1)]
    elif temp == 1:
        animals = ["pandas", "frogs", "fireflies", "kittens", "puppies", "parrots", "bunnies", "tropical fish"]
        prompt =  "oil painting of sks person, french moustache, heavy brushstrokes, beautiful eyes, surrounded by " + animals[random.randint(0, len(animals) - 1)] + ", interdimensional portal, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski"
    elif temp == 2:
        jobs = ["a chef", "a wizard", "a paladin", "a pianist", "a pilot"]
        prompt = "oil painting, portrait of sks person as " + jobs[random.randint(0 , len(jobs) - 1)] + ", heavy brushstrokes, beautiful eyes, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski"
    else:
        prompt = "oil painting of sks person, french moustache, heavy brushstrokes, beautiful eyes, "
        for _ in range(random.randint(1, 3)):
            prompt = prompt + middle[random.randint(0, len(middle) - 1)] + ", "
        prompt = prompt + "dramatic lighting, insanely detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski"                       
    
    # prompt editing: 50% of steps on sks person then 50% of steps on tree
    #prompt = "oil painting of [sks person:tree:0.5], oil portrait of sks person, french moustache, surrounded by puppies, black shades, puppies, colorful blazer, rainbow bowtie, boutonnière, heavy brushstrokes, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski"
    
    # prompt editing: 25 steps on sks person before switching to tree
    #prompt = "oil painting of [sks person:tree:25], oil portrait of sks person, french moustache, surrounded by puppies, black shades, puppies, colorful blazer, rainbow bowtie, boutonnière, heavy brushstrokes, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski"

    # prompt alternating
    # primary prompt goes first then secondary
    # a good example is [box|llama] - a llama that should take on the shape of a box
    #prompt = "[round|dog] painted by greg rutkowski"
    
    #prompt = "oil painting of sks person, oil portrait of sks person, french moustache, surrounded by cats, colorful blazer, rainbow bowtie, boutonnière, heavy brushstrokes, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski"
    prompt = "midjourneyart style"
    
    num_inference_steps=68
        
    print(f"{prompt}-{seed}")
    guidance_scale=7.5
    image = pipe(prompt, height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, generator=g_cuda).images[0]
    
    metadata = PngInfo()
    metadata.add_text("height", str(height))
    metadata.add_text("width", str(width))
    metadata.add_text("num_inference_steps", str(num_inference_steps))
    metadata.add_text("guidance_scale", str(guidance_scale))
    metadata.add_text("prompt", prompt)
    metadata.add_text("model_id", model_id)
    metadata.add_text("seed", str(seed))
    
    file_name = f"{uuid.uuid4().int & (1<<64)-1}"
    image.save(f"/home/jovyan/work/output/{file_name}.png", pnginfo=metadata)
    targetImage = PngImageFile(f"/home/jovyan/work/output/{file_name}.png")
    print(f"saved to /home/jovyan/work/output/{file_name}.png")
    #print("with metadata:", targetImage.text)
    print()
    #image.show()

## brute force v5 with no moustache prompts
use this with models trained on females

In [ ]:
height=512
width=512

#height=640
#width=640

#height=720
#width=720

#height=824
#width=824

#height = 720
#width = 1280
#width=1888

import random
from PIL.PngImagePlugin import PngImageFile, PngInfo
middle = ["royal gardens", "garden of eden", "eden", "mandelbulb fractal", "mandelbrot set", "antimatter", "gravity", "futuristic", "robotic", "photoshopped", \
          "bright neon colors", "glacier", "glaciation", "aesthetically pleasing", "fireworks", "visually stunning", "heavenly glow", "heavenly aura", \
          "4k resolution", "8k resolution", "4k res", "8k res", "assassin's creed costume", "Thermoluminescence", "aztec god", "psychedelic rainbow magic", \
          "sunlight streaming through clouds", "wispy clouds", "long hair", "long hairstyle", "bald", "wavy hair", "this is fine meme dog", "icy", "snow", \
          "nuclear mushroom", "a million colourful balloons", "evga FTW3 RTX 3090", "rapunzel hair", "flowers in hair", "macaw", "knightly armour", "shoe on head", \
          "book on head", "high-key", "longest moustache in the world", "casting a spell", "in varrock", "in lumbridge", "with a fire staff", "shoe on head", \
          "colourful bowtie", "colourful scarf", "colourful party hat", "osrs party hat", "runescape party hat", "rune scimitar", "osrs graphics", "2007scape", \
          "old master", "old masters", "louvre collection", "museum collection", "traditional chinese wedding dress", "Todos Santos Cuchumatán", "wearing Sari", \
          "wearing gho", "Conical hat", "Sardinian traditional dress", "keffiyeh", "shemagh" ,"ghutrah", "colourful traditional clothing", \
          "wearing long royal robe", "sequins", "academic", "jeweled", "detailed face", "fashion photography", "male model", "shallow depth of field", \
          "morrowind", "beautiful composition", "chin-strap facial hair", "colourful facial hair", "colourful hair", "painting in museum", "sharp focus", \
          "realistic", "intricate", "octane render", "trending artstation", "monochrome", "extreme detail", "Utrecht Caravaggism", "on a magic carpet", \
          "on horseback", "glitter", "neon lights", "long hair", "inferno", "vivid colors", "mycelium", "colourful smoke", "colourful clouds", \
          "quantum physics", "rembrandt lighting", "tesla coil", "plasma", "happy accidents", "tekken character", "street fighter character", \
          "atari graphics", "tempera", "Ukiyo-e", "cumulus", "portrait", "being fed grapes", "looking at camera", "isometric view", "full body", \
          "from behind", "aerial shot", "bird's eye shot", "😂", "❤️", "🤣", "🔥", "😍", "🥰", "🙏", "✨", "🔞💦🍆😳👄🍑🚿😈👅", "World Heritage Site", \
          "sunbeam", "crepuscular rays", "god rays", "asperitas clouds", "かわいい", "oneshota", "Dungeons & Dragons", "patronus", "Lovecraftian", "daguerreotype", \
          "Gorillaz", "prismatic", "bokeh", "raining", "far from camera", "vibrant", "epic", "volumetric lighting", "jellyfish from finding nemo", "disney", "pixar", \
          "3D render", "boutonnière", "balloons from UP", "HDR", "holi", "turban", "with a thousand arms", "mushroom helmet", "reflection", "transparent", \
          "british explorers", "smoking a cigar", "drinking a cocktail", "summoning a taxi", "german traditional dress", "santa hat", "noita", "Cel Shaded", \
          "Margit, the Fell Omen", "Morgott, the Omen King", "bandholz beard", "Rennala, Queen of the Full Moon", "elden beast", "elden ring", "fel fire", \
          "blue fire", "smooth", "taniwha", "surrounded by sheep", "Silver fern", "tui", "wood pigeon", "kereru", "pounamu", "kiwiana", "pohutawaka", \
          "kowhai", "surrounded by new zealand birds", "surrounded by new zealand bush", "surrounded by clouds", "sharp focus", "Picasso's African Period", \
          "Picasso's Blue Period", "fine art", "bladerunner", "baroque element", "baroque elements", "complimentry colours", "river", "surrounded by butterflies", \
          "surrounded by flowers", "unreal engine 5", "lightsaber", "hacker", "bowtie", "Barong", "balinese mask", "tribal mask", "african mask", \
          "award winning photography", "award winning", "deviantart", "surrounded by ice", "frozen", "divine", "surrounded by clifford the big red dog", \
          "monocle", "top hat", "meme", "kimono", "Windsor glasses", "surrounded by bunnies", "surrounded by piglets", "surrounded by pandas", \
          "surrounded by frogs", "blindfolded", "gold chain", "bling", "surrounded by puppies", "surrounded by dogs", "surrounded by kittens", \
          "surrounded by cats", "LSD visuals", "DMT fractal patterns", "ornate", "hallucinogen", "mutagen", "mutated", "coronation robes 1701", \
          "masterpiece", "fu manchu moustache", "star-burns", "muttonchops", "captain hook moustache", "salvador dali moustache", "animorphs", \
          "on fire", "barbaric", "hyper detailed", "ultra sharp", "sunny", "overwatch character", "highly detailed", "detailed face", "papakha hat", \
          "papakha woolen hat", "tamoko", "cinematic lighting", "golden", "rich", "exotic", "spicy", "fruity", "yoked", "jacked", "holy", "royal", \
          "bearded", "neon", "waterfall", "whirlpool", "electric", "cyberpunk", "futuristic", "pink", "opal", "iridescent", "vibrant", "high society", \
          "artistic", "artstation", "rainbow supernova", "masculine", "rule of thirds", "beautiful", "8k", "4k", "hooded cloak", "glowing", "french moustache", \
          "viking beard", "handlebar moustache", "imperial beard", "Ultra realistic", "heavy brushstrokes", "crisp", "sharp", "ultra sharp", "high resolution", \
          "breathtaking", "trending on artstation", "bioluminous", "funky", "medusa", "supernova", "nebula", "AI", "vibrant", "white dreadlocks", "rainbow", \
          "cotton candy", "popcorn", "coral reef", "tropical fish", "fish and chips"]

prompts = []
prompts.append("oil painting of (sks person), heavy brushstrokes, dramatic lighting, very detailed, insane detail, trending on artstation, 4k, craig mullins, greg rutkowski")
prompts.append("oil painting of sks person, heavy brushstrokes, beautiful eyes, surrounded by exploding nebula, dramatic lighting, very detailed, insanely detailed, trending on artstation, 4k, craig mullins, greg rutkowski")
prompts.append("oil painting of sks person, heavy brushstrokes, beautiful eyes, surrounded by exploding nebula, interdimensional, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski")
prompts.append("oil painting of sks person, heavy brushstrokes, beautiful eyes, whirlpool, surrounded by exploding nebula, interdimensional, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski")
prompts.append("oil painting of sks person, heavy brushstrokes, beautiful eyes, whirlpool, surrounded by exploding nebula, pink, interdimensional, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski")
prompts.append("oil painting of sks person, heavy brushstrokes, beautiful eyes, surrounded by exploding nebula, angelic, gates of heaven, interdimensional, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski")
prompts.append("oil painting of sks person, heavy brushstrokes, beautiful eyes, surrounded by exploding nebula, demonic, gates of hell, interdimensional, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski")
prompts.append("oil painting of sks person, heavy brushstrokes, beautiful eyes, surrounded by heaven and hell, interdimensional portal, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski")
prompts.append("oil painting portrait of sks person, heavy brushstrokes, beautiful eyes, whirlpool, surrounded by exploding nebula, interdimensional, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski")
prompts.append("oil painting portrait of sks person, surrounded by colorful puppies, colorful blazer, beautiful brown eyes, heavy brushstrokes, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski")
prompts.append("oil painting portrait of sks person, surrounded by colorful cows, colorful blazer, beautiful brown eyes, heavy brushstrokes, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski")

# use GOATED "jeremy-midjourney-style-2500-v4" with this loop 
while True:
    # can set seed here
    seed = uuid.uuid4().int & (1<<64)-1
    g_cuda = torch.Generator("cuda").manual_seed(seed)
    
    prompt = "Hello world!"

    temp = random.randint(0, 3)
    if temp == 0:
        prompt = prompts[random.randint(0, len(prompts) - 1)]
    elif temp == 1:
        animals = ["pandas", "frogs", "fireflies", "kittens", "puppies", "parrots", "bunnies", "tropical fish"]
        prompt =  "oil painting of sks person, heavy brushstrokes, beautiful eyes, surrounded by " + animals[random.randint(0, len(animals) - 1)] + ", interdimensional portal, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski"
    elif temp == 2:
        jobs = ["a model", "an author", "an artist", "a writer", "a queen", "a princess", "a maid", "a nurse"]
        prompt = "oil painting, portrait of sks person as " + jobs[random.randint(0 , len(jobs) - 1)] + ", heavy brushstrokes, beautiful eyes, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski"
    else:
        prompt = "oil painting of sks person, heavy brushstrokes, beautiful eyes, "
        for _ in range(random.randint(1, 3)):
            prompt = prompt + middle[random.randint(0, len(middle) - 1)] + ", "
        prompt = prompt + "dramatic lighting, insanely detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski"                       
    
    # prompt editing: 50% of steps on sks person then 50% of steps on tree
    #prompt = "oil painting of [sks person:tree:0.5], oil portrait of sks person, french moustache, surrounded by puppies, black shades, puppies, colorful blazer, rainbow bowtie, boutonnière, heavy brushstrokes, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski"
    
    # prompt editing: 25 steps on sks person before switching to tree
    #prompt = "oil painting of [sks person:tree:25], oil portrait of sks person, french moustache, surrounded by puppies, black shades, puppies, colorful blazer, rainbow bowtie, boutonnière, heavy brushstrokes, dramatic lighting, very detailed, intricate, trending on artstation, 4k, craig mullins, greg rutkowski"

    # prompt alternating
    # primary prompt goes first then secondary
    # a good example is [box|llama] - a llama that should take on the shape of a box
    #prompt = "[round|dog] painted by greg rutkowski"
    
    prompt = "midjourneyart style"
    
    num_inference_steps=68
        
    print(f"{prompt}-{seed}")
    guidance_scale=7.5
    image = pipe(prompt, height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, generator=g_cuda).images[0]
    
    metadata = PngInfo()
    metadata.add_text("height", str(height))
    metadata.add_text("width", str(width))
    metadata.add_text("num_inference_steps", str(num_inference_steps))
    metadata.add_text("guidance_scale", str(guidance_scale))
    metadata.add_text("prompt", prompt)
    metadata.add_text("model_id", model_id)
    metadata.add_text("seed", str(seed))
    
    file_name = f"{uuid.uuid4().int & (1<<64)-1}"
    image.save(f"/home/jovyan/work/output/{file_name}.png", pnginfo=metadata)
    targetImage = PngImageFile(f"/home/jovyan/work/output/{file_name}.png")
    print(f"saved to /home/jovyan/work/output/{file_name}.png")
    print()
    image.show()

## custom dimensions single image synthesis

couldn't quite make it to 1024 x 1024 so found the maximum resolution my machine is capable of at 1008x1008 (1016x1016) does not work

might need to restart kernel first for large resolutions

In [ ]:
import string
import random

height = 720
width = 1320

f = open("4letterwords.txt", "r")
obj = f.read()
four_letter_words = obj.split("\n")

while True:
    # can set seed here
    seed = uuid.uuid4().int & (1<<64)-1
    seed = 12751836622659959420
    g_cuda = torch.Generator("cuda").manual_seed(seed)

    prompt = "realism painting of attractive ((sks jeremy)), handlebar moustache, opal, "
    prompt = prompt + four_letter_words[random.randint(0, len(four_letter_words) - 1)]
    prompt = prompt + " in the style of jakub rozalski"

    num_inference_steps=99

    print(f"{prompt}-{seed}")
    image = pipe(prompt, height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=7.5, generator=g_cuda).images[0]

    image.save(f"/home/jovyan/work/output/{prompt}-{seed}-{num_inference_steps}.png")
    print(f"saved to /home/jovyan/work/output/")
    print()
image

## 3x3 grid image synthesis

In [ ]:
prompt = "portrait of (((((sks jeremy))))), iridescent magic, rainbow universe, exploding supernova, medusa in the style of Greg Rutkowski and Wlop"
num_samples = 12
guidance_scale = 7.5
num_inference_steps = 68
height = 512
width = 512

# default no seed
g_cuda = None

# can set seed here
seed = uuid.uuid4().int & (1<<64)-1
g_cuda = torch.Generator("cuda").manual_seed(seed)

images = pipe([prompt] * num_samples, height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, generator=g_cuda).images

num_cols = 3
num_rows = 4

all_images = []
all_images.extend(images)
grid = image_grid(all_images, rows=num_rows, cols=num_cols)
print("prompt:\n " + prompt + "\nseed:\n ", str(seed))

grid.save(f"/home/jovyan/work/output/{prompt}-{seed}.png")
grid

## experimental 6x6 grid

In [ ]:
prompt = "a painting of sks jeremy in the style of greg rutkowski"
num_samples = 36
guidance_scale = 7.5
num_inference_steps = 50
height = 256
width = 256
g_cuda = None

images = pipe([prompt] * num_samples, height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, generator=g_cuda).images

num_cols = 6
num_rows = 6

all_images = []
all_images.extend(images)
grid = image_grid(all_images, rows=num_rows, cols=num_cols)
print("prompt:", prompt)
grid

## experimental 9x9 grid inference

In [ ]:
prompt = "a painting of sks jeremy in the style of greg rutkowski"
num_samples = 81
guidance_scale = 7.5
num_inference_steps = 50
height = 128
width = 128
g_cuda = None

images = pipe([prompt] * num_samples, height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, generator=g_cuda).images

num_cols = 9
num_rows = 9

all_images = []
all_images.extend(images)
grid = image_grid(all_images, rows=num_rows, cols=num_cols)
print("prompt:", prompt)
grid

## experimental 18x18 grid inference

In [ ]:
prompt = "a painting of sks jeremy in the style of greg rutkowski"
num_samples = 324
guidance_scale = 7.5
num_inference_steps = 50
height = 64
width = 64
g_cuda = None

images = pipe([prompt] * num_samples, height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, generator=g_cuda).images

num_cols = 18
num_rows = 18

all_images = []
all_images.extend(images)
grid = image_grid(all_images, rows=num_rows, cols=num_cols)
print("prompt:", prompt)
grid

## md vs sd in terms of landscapes

In [ ]:
num_inference_steps=68
guidance_scale=7.5
width=512
height=512
ideas = ["spacestation", "cliff", "landscape", "field", "river", "city", "lookout", "river", "lake", "mountain", "nebula", "black hole", "library", "universe", "supernova"]

for i in range(len(ideas) - 1):

    prompt = "oil painting of a "
    prompt = prompt + ideas[i]
    prompt = prompt + ", midjourneyart style"

    seed = uuid.uuid4().int & (1<<64)-1
    seed = 420

    g_cuda = torch.Generator("cuda").manual_seed(seed)

    print(f"{prompt}-{seed}")
    image = pipe(prompt, height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]
    image.save(f"/home/jovyan/work/output/{prompt}-{seed}-md.png")

# set variables for inference
model_id = "/home/jovyan/work/models/sd-v1-4-full-ema/"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

# to disable the safety checker
pipe.safety_checker = lambda images, clip_input: (images, False)

for i in range(len(ideas) - 1):

    prompt = "oil painting of a "
    prompt = prompt + ideas[i]
    prompt = prompt + ", midjourneyart style"

    seed = uuid.uuid4().int & (1<<64)-1
    seed = 420

    g_cuda = torch.Generator("cuda").manual_seed(seed)

    print(f"{prompt}-{seed}")
    image = pipe(prompt, height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]
    image.save(f"/home/jovyan/work/output/{prompt}-{seed}-sd.png")

## md-jeremy vs jeremy-2500 in terms of sks jeremy

In [ ]:
num_inference_steps=68
guidance_scale=7.5
height=720
width=1320

# set variables for inference
model_id = "/home/jovyan/work/models/jeremy-2500/"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

prompt = "oil painting of sks jeremy, midjourneyart style"

seed = 415
for _ in range(10):

    g_cuda = torch.Generator("cuda").manual_seed(seed)
    
    seed = seed + 1

    print(f"{prompt}-{seed}")
    image = pipe(prompt, height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]
    image.save(f"/home/jovyan/work/output/{prompt}-{seed}-jrs.png")

# set variables for inference
model_id = "/home/jovyan/work/models/jeremy-midjourney-2500/"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

# to disable the safety checker
pipe.safety_checker = lambda images, clip_input: (images, False)

seed = 415
for _ in range(10):

    g_cuda = torch.Generator("cuda").manual_seed(seed)
    
    seed = seed + 1

    print(f"{prompt}-{seed}")
    image = pipe(prompt, height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]
    image.save(f"/home/jovyan/work/output/{prompt}-{seed}-md-jrs.png")

## variations

can take a seed, prompt and model and create variations of an input image - looks like a great way to make a video too

In [ ]:
import random
import uuid
import torch
from PIL.PngImagePlugin import PngImageFile, PngInfo
from diffusers import StableDiffusionPipeline
targetImage = PngImageFile("/home/jovyan/work/potential.png")
print('metadata: ', targetImage.text)

model_id = ""

if model_id != targetImage.text["model_id"]:
    print("Warning! model_id mixmatch:")
    print(targetImage.text["model_id"] + " does not match current model " + model_id)
    print("now loading " + targetImage.text["model_id"])
    
    model_id = targetImage.text["model_id"]
    model_dir = "/home/jovyan/work/models/" + model_id + "/"
    pipe = StableDiffusionPipeline.from_pretrained(model_dir, torch_dtype=torch.float16).to("cuda")

height = int(targetImage.text["height"])
width = int(targetImage.text["width"])

prompt = targetImage.text["prompt"]

while True:
    seed = int(targetImage.text["seed"])
    g_cuda = torch.Generator("cuda").manual_seed(seed)
    
    # METHOD ONE
    # replace some of the prompt with random characters
    # will not replace whitespace to ensure tokens are kept separate
    roman = "abcdefghijklmnopqrstuvwxyz"
    new_prompt = ""
    new_prompt = list(prompt)
    for _ in range(random.randint(1, 5)):
        index = random.randint(0, len(new_prompt) - 1)
        while new_prompt[index] == " " or new_prompt[index] == ",":
            index = random.randint(0, len(new_prompt) - 1)
        new_prompt[index] = roman[random.randint(0, len(roman) - 1)]
    new_prompt = "".join(new_prompt)
    
    # METHOD TWO
    # replace a random " " with "  "
    #
    # i dont actually know how to code this lol
    
    num_inference_steps = targetImage.text["num_inference_steps"]
    num_inference_steps = int(num_inference_steps)
        
    print(f"{new_prompt}-{seed}-{num_inference_steps}")
    guidance_scale=targetImage.text["guidance_scale"]
    guidance_scale=float(guidance_scale)
    image = pipe(new_prompt, height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, generator=g_cuda).images[0]
    
    metadata = PngInfo()
    metadata.add_text("height", str(height))
    metadata.add_text("width", str(width))
    metadata.add_text("num_inference_steps", str(num_inference_steps))
    metadata.add_text("guidance_scale", str(guidance_scale))
    metadata.add_text("prompt", new_prompt)
    metadata.add_text("model_id", model_id)
    metadata.add_text("seed", str(seed))
    
    file_name = f"{uuid.uuid4().int & (1<<64)-1}"
    image.save(f"/home/jovyan/work/output/{file_name}.png", pnginfo=metadata)
    targetImage = PngImageFile(f"/home/jovyan/work/output/{file_name}.png")
    print(f"saved to /home/jovyan/work/output/{file_name}.png")
    print("with metadata:", targetImage.text)
    print()
    image.show()